In [1]:
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
from tqdm import tqdm_notebook

### Connecting to Impala

In [12]:
# Establishing connection

# conn = connect(port = 2324, database = 'cdr')
conn = connect(host = 'bdl3.eng.tau.ac.il', database = 'cdr')

# Creating cursor
cur = conn.cursor()

In [14]:
for t in tqdm_notebook(range(10,20)):
    # Getting a sample of unchecked imsi
    # Getting the imsi list
    imsi_list = pd.read_csv('../data/imsi_filtered.csv')

    # Getting the checked imsi list
    checked_imsi = pd.read_csv('../data/checked_imsi.csv')

    # The list of unchecked imsi
    unchecked_imsi = imsi_list[~imsi_list.imsi.isin(checked_imsi.imsi.unique())]

    # Getting a sample of unchecked imsi
    sample_imsi = unchecked_imsi.sample(9999).imsi.values

    # Creating a string of the imsi sapmle for the query
    sample_str = str(tuple(sample_imsi))
    
    # Creating the query
    query = 'SELECT imsi, date_stamp, time_stamp, halfhouridx, latitude, longtitude FROM traces WHERE imsi in ' + sample_str
    
    # Executing the query
    cur.execute(query)
    data = as_pandas(cur)
    
    # Save the data
    data.to_csv(f'../data/samples/sample_{t}.csv', index=False)
    
    # Update the checked imsi list
    checked_imsi = pd.concat([checked_imsi, imsi_list[imsi_list.imsi.isin(sample_imsi)]])
    checked_imsi.to_csv('../data/checked_imsi.csv', index=False)

A Jupyter Widget

In [2]:
checked_imsi = pd.read_csv('../data/checked_imsi.csv')

In [4]:
print(f'{checked_imsi.shape[0]:,d}')
print(f'{checked_imsi.total_records.sum():,d}')

211,443
3,184,043,431
